In [1]:
import numpy as np 
import pandas as pd 
from datetime import datetime, date, timedelta
import os
import pymssql as mssql
import json
from multiprocessing import Pool    
from pathlib import Path
from functools import partial 
import logging, traceback
import sys
from tqdm import tqdm_notebook as tqdm
sys.path.insert(0, 'D:\庫存健診開發\code\monthly\core')


In [2]:
import ALL_STOCK_preprocess_function as preprocess

In [3]:
d = preprocess.stock_query(date(2007,7,2), date(2019,9,30))

In [4]:
stock, index, industry_index = d[0], d[1], d[2]

In [5]:
df_list = [group[1] for group in stock.groupby(stock['StockNo'])]
timedf = pd.DataFrame(stock['ts'].unique(), columns=['ts'])
timedf = timedf.sort_values(by='ts')
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for _, x in enumerate(tqdm(pool.imap_unordered(partial(preprocess.FillMissingTime, timedf=timedf, end_date=date(2019,9,30)), df_list), total=len(df_list)), 1):
                if x[0]:
                    output_list.append(x[1])
                else:
                    continue
                    
df = pd.concat(output_list)

In [6]:
df

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,investment_buy,dealer_buy,foreign_ratio,investment_ratio,dealer_ratio,corporation_ratio,industry,On_Date,Restart_date,eliminate
0,2007-07-02,1103,¹Åªd,24.5,24.7,24.00,24.20,5563879.0,135167.0,6737.0,...,-30.0,64.0,2.61,0.21,0.58,3.40,None,NaT,NaT,0.0
1,2007-07-03,1103,¹Åªd,24.4,24.5,23.70,24.00,8736835.0,210146.0,6737.0,...,0.0,99.0,2.41,0.21,0.59,3.21,None,NaT,NaT,0.0
2,2007-07-04,1103,¹Åªd,23.7,24.5,23.70,23.85,7687399.0,184886.0,6737.0,...,-9.0,133.0,2.45,0.21,0.61,3.27,None,NaT,NaT,0.0
3,2007-07-05,1103,¹Åªd,24.2,24.6,23.85,24.30,12799095.0,310876.0,6737.0,...,139.0,-973.0,2.41,0.23,0.47,3.11,None,NaT,NaT,0.0
4,2007-07-06,1103,¹Åªd,24.8,26.0,24.70,26.00,27157762.0,687435.0,6737.0,...,144.0,-240.0,2.33,0.25,0.43,3.01,None,NaT,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2853,2019-09-23,9958,¥@¬ö¿û,75.8,76.5,75.40,75.80,1530090.0,116146.0,2093.0,...,0.0,-114.0,2.68,1.30,1.40,5.38,10,2008-03-12,NaT,0.0
2854,2019-09-24,9958,¥@¬ö¿û,75.8,78.2,75.60,77.00,4139948.0,319446.0,2093.0,...,-141.0,191.0,2.66,1.23,1.49,5.38,10,2008-03-12,NaT,0.0
2855,2019-09-25,9958,¥@¬ö¿û,76.5,77.3,76.30,76.80,1560688.0,119803.0,2093.0,...,-40.0,-141.0,2.25,1.21,1.42,4.88,10,2008-03-12,NaT,0.0
2856,2019-09-26,9958,¥@¬ö¿û,77.1,77.6,75.10,75.30,3547150.0,270866.0,2093.0,...,0.0,-84.0,2.09,1.21,1.38,4.68,10,2008-03-12,NaT,0.0


In [7]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for x in pool.imap_unordered(partial(preprocess.merge_index, index=index, industry_index=industry_index), df_list):
                output_list.append(x)

df = pd.concat(output_list)

In [8]:
import VWAP_feature_function_rolling_week as FeatureEngineering

In [9]:
with open(r'D:\庫存健診開發\code\monthly\config\columns_dict.json', 'r') as fp:
    columns_dict = json.load(fp)

In [9]:
stock_list = ['1101',
'1102',
'1216',
'1301',
'1303',
'1326',
'1402',
'1476',
'2002',
'2105',
'2207 ',
'2301 ',
'2303 ',
'2308 ',
'2311 ',
'2317 ',
'2324 ',
'2325 ',
'2330 ',
'2354 ',
'2357 ',
'2382 ',
'2395 ',
'2408 ',
'2409 ',
'2412 ',
'2474 ',
'2801 ',
'2880 ',
'2881 ',
'2882 ',
'2883 ',
'2884 ',
'2885 ',
'2886 ',
'2887 ',
'2890 ',
'2891 ',
'2892 ',
'2912 ',
'3008 ',
'3045 ',
'3474 ',
'3481 ',
'4904 ',
'4938 ',
'5880 ',
'6505 ',
'9904 ',
'1227 ',
'1262 ',
'1319 ',
'1440 ',
'1477 ',
'1504 ',
'1536 ',
'1589 ',
'1590 ',
'1605 ',
'1704 ',
'1707 ',
'1717 ',
'1722 ',
'1723 ',
'1789 ',
'1802 ',
'2015 ',
'2049 ',
'2059 ',
'2101 ',
'2103 ',
'2106 ',
'2201 ',
'2204 ',
'2206 ',
'2231 ',
'2313 ',
'2327 ',
'2345 ',
'2347 ',
'2352 ',
'2353 ',
'2355 ',
'2356 ',
'2360 ',
'2362 ',
'2376 ',
'2377 ',
'2379 ',
'2383 ',
'2385 ',
'2392 ',
'2448 ',
'2449 ',
'2451 ',
'2498 ',
'2501 ',
'2542 ',
'2603 ',
'2609 ',
'2610 ',
'2615 ',
'2618 ',
'2707 ',
'2809 ',
'2812 ',
'2823 ',
'2834 ',
'2845 ',
'2849 ',
'2867 ',
'2888 ',
'2903 ',
'2915 ',
'3034 ',
'3037 ',
'3044 ',
'3189 ',
'3231 ',
'3673 ',
'3682 ',
'3702 ',
'4958 ',
'5264 ',
'5522 ',
'5871 ',
'6176 ',
'6239 ',
'6269 ',
'6285 ',
'6414 ',
'6415 ',
'6452 ',
'6456 ',
'8150 ',
'8454 ',
'8464 ',
'9907 ',
'9910 ',
'9914 ',
'9917 ',
'9921 ',
'9933 ',
'9938 ',
'9945 ']


In [12]:
df = df[df['StockNo'].isin(stock_list)]
df

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,index_open,index_high,index_low,index_close,index_vol,industry_open,industry_high,industry_low,industry_close,industry_vol
0,2007-07-02,1101,¥xªd,38.20,38.20,37.00,37.50,10222329.0,381140.0,32074.0,...,8903.77,8939.49,8849.35,8939.49,5490199.0,138.17,138.94,137.36,138.93,195301.0
1,2007-07-03,1101,¥xªd,37.70,37.80,37.05,37.40,9831445.0,367582.0,32074.0,...,8983.24,9019.93,8977.09,8996.20,6604477.0,139.64,140.89,139.54,140.88,218190.0
2,2007-07-04,1101,¥xªd,37.50,38.40,37.35,37.70,11124197.0,420368.0,32074.0,...,9069.32,9112.74,9034.50,9068.98,7261940.0,141.76,143.09,141.59,142.25,256483.0
3,2007-07-05,1101,¥xªd,37.70,38.00,37.00,37.00,15043469.0,559331.0,32074.0,...,9114.06,9148.78,9086.19,9148.78,7065444.0,142.52,143.50,141.42,142.47,227751.0
4,2007-07-06,1101,¥xªd,37.00,38.65,36.75,38.55,25513663.0,959258.0,32074.0,...,9169.56,9199.66,9118.26,9188.31,7514284.0,143.24,143.52,141.51,142.22,198150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024,2019-09-23,9945,¼í®õ·s,42.05,42.25,41.90,41.95,1674736.0,70366.0,10033.0,...,10924.81,10948.19,10905.92,10919.02,3976566.0,281.83,282.66,281.38,281.63,23216.0
3025,2019-09-24,9945,¼í®õ·s,41.95,42.30,41.90,42.15,1117767.0,47171.0,10033.0,...,10917.44,10947.53,10886.62,10918.01,3872482.0,281.61,282.79,281.38,281.61,27874.0
3026,2019-09-25,9945,¼í®õ·s,42.15,42.15,41.85,41.95,766944.0,32205.0,10033.0,...,10885.67,10885.67,10825.85,10873.69,3880869.0,281.62,284.06,281.25,282.49,42179.0
3027,2019-09-26,9945,¼í®õ·s,41.90,42.30,41.85,41.85,795199.0,33365.0,10033.0,...,10909.57,10934.91,10856.96,10871.99,3878757.0,282.82,284.79,282.32,282.66,42781.0


In [11]:
stock_list

['1101',
 '1102',
 '1216',
 '1301',
 '1303',
 '1326',
 '1402',
 '1476',
 '2002',
 '2105',
 '2207',
 '2301',
 '2303',
 '2308',
 '2311',
 '2317',
 '2324',
 '2325',
 '2330',
 '2354',
 '2357',
 '2382',
 '2395',
 '2408',
 '2409',
 '2412',
 '2474',
 '2801',
 '2880',
 '2881',
 '2882',
 '2883',
 '2884',
 '2885',
 '2886',
 '2887',
 '2890',
 '2891',
 '2892',
 '2912',
 '3008',
 '3045',
 '3474',
 '3481',
 '4904',
 '4938',
 '5880',
 '6505',
 '9904',
 '1227',
 '1262',
 '1319',
 '1440',
 '1477',
 '1504',
 '1536',
 '1589',
 '1590',
 '1605',
 '1704',
 '1707',
 '1717',
 '1722',
 '1723',
 '1789',
 '1802',
 '2015',
 '2049',
 '2059',
 '2101',
 '2103',
 '2106',
 '2201',
 '2204',
 '2206',
 '2231',
 '2313',
 '2327',
 '2345',
 '2347',
 '2352',
 '2353',
 '2355',
 '2356',
 '2360',
 '2362',
 '2376',
 '2377',
 '2379',
 '2383',
 '2385',
 '2392',
 '2448',
 '2449',
 '2451',
 '2498',
 '2501',
 '2542',
 '2603',
 '2609',
 '2610',
 '2615',
 '2618',
 '2707',
 '2809',
 '2812',
 '2823',
 '2834',
 '2845',
 '2849',
 '2867',
 

In [10]:
stock_list = [stock_no[:4] for stock_no in stock_list]

In [13]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for _, x in enumerate(tqdm(pool.imap_unordered(partial(FeatureEngineering.get_label, forward=20, look_back_month=3), df_list), total=len(df_list)), 1):
            output_list.append(x)

df = pd.concat(output_list, axis=0) 

In [31]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

for d in df_list:
    d['industry_high_month'] = d['industry_high'].rolling(window=20).max()
    d['industry_low_month'] = d['industry_low'].rolling(window=20).min()
    d['industry_open_month'] = d['industry_open'].shift(20)
    d['industry_vol_month'] = d['industry_vol'].rolling(window=20).sum()
    output_list.append(d)

df = pd.concat(output_list, axis=0)

In [11]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for _, x in enumerate(tqdm(pool.imap_unordered(partial(FeatureEngineering.separate_engineering, columns_dict=columns_dict, look_back=20, forward=20), df_list), total=len(df_list)), 1):
            output_list.append(x)

df = pd.concat(output_list, axis=0) 

In [9]:
df.to_csv(r'D:\庫存健診開發\data\month_processed.csv', index=False)

In [16]:
original = pd.read_csv(r'D:\庫存健診開發\data\month_processed.csv', converters={'ts': str, 'StockNo': str})

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
original['ts'] = pd.to_datetime(original['ts'])
df_new = pd.merge(df[['ts', 'StockNo', 'Y_TR', 'index_open_month', 'index_high_month', 'index_low_month', 'index_vol_month', 'industry_open_month', 'industry_high_month', 'industry_low_month', 'industry_vol_month']], original, on=['ts', 'StockNo'], how='inner')

In [41]:
len(df[df['Y_TR'] == 1])/len(df)

0.5015177294968085

In [35]:
for col in df_new.columns:
    if len(df_new[df_new[col].isnull()]) != 0:
        print(col)

In [7]:
df = df.fillna(0)

In [25]:
feature = FeatureEngineering.read_feature_list(r'D:\庫存健診開發\code\monthly\config\feature_dict.json', requirement='whole')

In [36]:
feature = ['total',
 'capital',
 'index_open_month',
 'index_high_month',
 'index_low_month',
 'index_close',
 'index_return',
 'index_vol_month',
 'industry_return',
 'foreign_ratio',
 'investment_ratio',
 'corporation_ratio',
 'close_return',
 'VWAP_return',
 'index_VWAP_slope',
 'VWAP_day20_return',
 'industry_VWAP_slope',
 'index_close_slope',
 'industry_close_slope',
 'index_close_ratio_day5',
 'index_vol_ratio_day5',
 'index_return_ratio_day5',
 'industry_close_ratio_day5',
 'industry_vol_ratio_day5',
 'industry_return_ratio_day5',
 'VWAP_return_ratio_day5',
 'close_return_ratio_day5',
 'vol_ratio_day5',
 'VWAP_ratio_day5',
 'VWAP_day20_ratio_day5',
 'VWAP_day20_return_ratio_day5',
 'index_close_ratio_day10',
 'index_vol_ratio_day10',
 'index_return_ratio_day10',
 'industry_close_ratio_day10',
 'industry_vol_ratio_day10',
 'industry_return_ratio_day10',
 'VWAP_return_ratio_day10',
 'close_return_ratio_day10',
 'vol_ratio_day10',
 'VWAP_ratio_day10',
 'VWAP_day20_ratio_day10',
 'VWAP_day20_return_ratio_day10',
 'index_close_ratio_day15',
 'index_vol_ratio_day15',
 'index_return_ratio_day15',
 'industry_close_ratio_day15',
 'industry_vol_ratio_day15',
 'industry_return_ratio_day15',
 'VWAP_return_ratio_day15',
 'close_return_ratio_day15',
 'vol_ratio_day15',
 'VWAP_ratio_day15',
 'VWAP_day20_ratio_day15',
 'VWAP_day20_return_ratio_day15',
 'index_close_ratio_day20',
 'index_vol_ratio_day20',
 'index_return_ratio_day20',
 'industry_close_ratio_day20',
 'industry_vol_ratio_day20',
 'industry_return_ratio_day20',
 'VWAP_return_ratio_day20',
 'close_return_ratio_day20',
 'vol_ratio_day20',
 'VWAP_ratio_day20',
 'VWAP_day20_ratio_day20',
 'VWAP_day20_return_ratio_day20',
 'index_close_momentum_day5',
 'index_vol_momentum_day5',
 'index_return_momentum_day5',
 'industry_close_momentum_day5',
 'industry_vol_momentum_day5',
 'industry_return_momentum_day5',
 'VWAP_return_momentum_day5',
 'close_return_momentum_day5',
 'vol_momentum_day5',
 'VWAP_momentum_day5',
 'VWAP_day20_momentum_day5',
 'VWAP_day20_return_momentum_day5',
 'index_close_momentum_day10',
 'index_vol_momentum_day10',
 'index_return_momentum_day10',
 'industry_close_momentum_day10',
 'industry_vol_momentum_day10',
 'industry_return_momentum_day10',
 'VWAP_return_momentum_day10',
 'close_return_momentum_day10',
 'vol_momentum_day10',
 'VWAP_momentum_day10',
 'VWAP_day20_momentum_day10',
 'VWAP_day20_return_momentum_day10',
 'index_close_momentum_day15',
 'index_vol_momentum_day15',
 'index_return_momentum_day15',
 'industry_close_momentum_day15',
 'industry_vol_momentum_day15',
 'industry_return_momentum_day15',
 'VWAP_return_momentum_day15',
 'close_return_momentum_day15',
 'vol_momentum_day15',
 'VWAP_momentum_day15',
 'VWAP_day20_momentum_day15',
 'VWAP_day20_return_momentum_day15',
 'index_close_momentum_day20',
 'index_vol_momentum_day20',
 'index_return_momentum_day20',
 'industry_close_momentum_day20',
 'industry_vol_momentum_day20',
 'industry_return_momentum_day20',
 'VWAP_return_momentum_day20',
 'close_return_momentum_day20',
 'vol_momentum_day20',
 'VWAP_momentum_day20',
 'VWAP_day20_momentum_day20',
 'VWAP_day20_return_momentum_day20',
 'vol_VWAP_corr',
 'vol_volatile_corr',
 'index_vol_close_corr',
 'index_vol_volatile_corr',
 'industry_vol_close_corr',
 'industry_vol_volatile_corr',
 'index_close_skew',
 'index_close_kurtosis',
 'index_vol_skew',
 'index_vol_kurtosis',
 'industry_close_skew',
 'industry_close_kurtosis',
 'industry_vol_skew',
 'industry_vol_kurtosis',
 'vol_skew',
 'vol_kurtosis',
 'VWAP_skew',
 'VWAP_kurtosis',
 'VWAP_day20_skew',
 'VWAP_day20_kurtosis',
 'foreign_buy_week_ratio',
 'foreign_buy_2week_ratio',
 'foreign_buy_3week_ratio',
 'foreign_buy_continuous',
 'foreign_buy_buy_day_ratio',
 'foreign_buy_sell_day_ratio',
 'investment_buy_week_ratio',
 'investment_buy_2week_ratio',
 'investment_buy_3week_ratio',
 'investment_buy_continuous',
 'investment_buy_buy_day_ratio',
 'investment_buy_sell_day_ratio',
 'index_pvt',
 'industry_pvt',
 'foreign_pvt',
 'investment_pvt',
 'pvt',
 'RSI_20',
 'SOk5',
 'SOd3',
 'SOk20',
 'SOd5',
 'industry',
 'ts',
 'StockNo']

In [37]:
# df['ts'] = pd.to_datetime(df['ts'])
train_df = df_new[df_new.ts.dt.date < date(2017,9,1)]
test_df = df_new[df_new.ts.dt.date >= date(2017,9,1)]

In [45]:
train_feature = feature + ['Y']
train_feature

['total',
 'capital',
 'index_open_month',
 'index_high_month',
 'index_low_month',
 'index_close',
 'index_return',
 'index_vol_month',
 'industry_return',
 'foreign_ratio',
 'investment_ratio',
 'corporation_ratio',
 'close_return',
 'VWAP_return',
 'index_VWAP_slope',
 'VWAP_day20_return',
 'industry_VWAP_slope',
 'index_close_slope',
 'industry_close_slope',
 'index_close_ratio_day5',
 'index_vol_ratio_day5',
 'index_return_ratio_day5',
 'industry_close_ratio_day5',
 'industry_vol_ratio_day5',
 'industry_return_ratio_day5',
 'VWAP_return_ratio_day5',
 'close_return_ratio_day5',
 'vol_ratio_day5',
 'VWAP_ratio_day5',
 'VWAP_day20_ratio_day5',
 'VWAP_day20_return_ratio_day5',
 'index_close_ratio_day10',
 'index_vol_ratio_day10',
 'index_return_ratio_day10',
 'industry_close_ratio_day10',
 'industry_vol_ratio_day10',
 'industry_return_ratio_day10',
 'VWAP_return_ratio_day10',
 'close_return_ratio_day10',
 'vol_ratio_day10',
 'VWAP_ratio_day10',
 'VWAP_day20_ratio_day10',
 'VWAP_day20_

In [46]:
train_df[train_feature].to_csv(r'D:\OneDrive - SinoPac\training\month_train_index.csv', index=False)

In [47]:
test_df[train_feature].to_csv(r'D:\OneDrive - SinoPac\training\month_test_index_labeled.csv', index=False)

In [27]:
len(test_df['StockNo'].unique())

928

In [29]:
len(train_df[train_df.Y == 1])/len(train_df)

0.48006947383848425

In [40]:
train_df[train_feature]

,total,capital,index_open_month,index_high_month,index_low_month,index_close,index_return,index_vol_month,industry_return,foreign_ratio,...,pvt,RSI_20,SOk5,SOd3,SOk20,SOd5,industry,ts,StockNo,Y_TR
0,708179000.0,32487.0,8903.77,9807.91,8977.09,9072.57,1.49,163482782.0,4.48,46.91,...,1.260994e+05,0.342664,0.165563,0.276419,0.165563,0.149586,0.0,2007-07-30,1101,1
1,753395000.0,32487.0,8983.24,9807.91,8996.81,9287.25,3.24,164062376.0,5.25,46.82,...,1.377989e+06,0.474590,0.443709,0.360064,0.443709,0.247640,0.0,2007-07-31,1101,0
2,853873000.0,32487.0,9069.32,9807.91,8891.88,8891.88,-1.95,165296559.0,1.10,46.38,...,1.051719e+06,0.434756,0.086093,0.223078,0.086093,0.193786,0.0,2007-08-01,1101,1
3,738356000.0,32487.0,9114.06,9807.91,8727.64,8950.57,-2.17,165395388.0,-0.21,46.08,...,5.036673e+05,0.363721,0.142857,0.182968,0.100559,0.162708,0.0,2007-08-02,1101,1
4,722089000.0,32487.0,9169.56,9807.91,8727.64,9057.82,-1.42,163746605.0,0.29,46.14,...,3.798632e+05,0.380502,0.609524,0.396246,0.357542,0.227655,0.0,2007-08-03,1101,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381618,87563000.0,13934.0,10480.67,10619.42,10225.28,10515.51,0.89,108851945.0,-1.66,12.49,...,-8.409451e+05,0.627338,0.848485,0.821769,0.909091,0.811147,20.0,2017-08-25,9945,0
381619,221047000.0,13934.0,10396.76,10619.42,10225.28,10525.98,0.95,110362489.0,-1.35,12.34,...,-4.912968e+05,0.700768,0.766667,0.794218,0.888889,0.837061,20.0,2017-08-28,9945,0
381620,147118000.0,13934.0,10408.53,10619.42,10225.28,10496.57,0.57,110786512.0,-2.55,12.29,...,-3.718994e+05,0.674077,0.352941,0.573580,0.825397,0.833173,20.0,2017-08-29,9945,0
381621,341438000.0,13934.0,10479.26,10619.42,10225.28,10569.40,0.48,111150711.0,-2.19,12.30,...,-1.372293e+05,0.680796,0.647059,0.610319,0.904762,0.857036,20.0,2017-08-30,9945,0
